In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import sent2vec

In [2]:
user_df=pd.read_csv('data/health_experts100.csv')
##id, tweet, user

In [6]:
##Clean up text to remove special characters
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
from string import digits
import re
irrelevant_chars="~?!./\:;+=&^%$#@(,)[]_*"
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def deep_clean(x):
    x=x.lower()
    x=re.sub(r'http\S+', '', x)
    remove_digits = str.maketrans(' ', ' ', digits)
    remove_chars = str.maketrans(' ', ' ', irrelevant_chars)
    x = x.translate(remove_digits)
    x = x.translate(remove_chars)
    x = emoji_pattern.sub(r'', x)
    x=x.replace("-",' ')
    x=x.replace('!',' ')
    x=x.replace('?',' ')
    x=x.replace('@',' ')
    x=x.replace('&',' ')
    x=x.replace('$',' ')
    x=x.replace('``',' ')
    x=x.replace("'s",' ')
    x=x.replace("''",' ')
    return x

In [7]:
##NOTE: Students will need to group individual tweets by the user. Here's a snippet:
user_text={}
for i in range(len(user_df)): #Here df is the tweet data.
    user=user_df['screen_name'].iloc[i]
    text=user_df['text'].iloc[i]
    if user not in user_text:
        user_text[user]=' '
    user_text[user]=deep_clean(str(user_text[user])+' '+str(text))
user_text_df=pd.DataFrame(user_text.items(),columns=['screen_name','cleaned_text'])

In [8]:
#https://github.com/epfml/sent2vec -> download sent2vec_twitter_bigrams
model = sent2vec.Sent2vecModel()
model.load_model('twitter_bigrams.bin')

embeddings = model.embed_sentences(user_text_df['cleaned_text'].tolist())
e=embeddings.tolist()

user_text_df['embeddings']=e

In [9]:
filename = 'model.sav'
model=pickle.load(open(filename, 'rb'))

/Users/paiheng/miniconda3/envs/General/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
y_pred=model.predict(np.asarray(user_text_df['embeddings'].tolist()))

In [ ]:
y_pred